In [3]:
import sys
import soundfile as sf
import librosa
import os
from os import path
from os import listdir
from os.path import isfile, isdir, join
import numpy as np
import scipy.io as spio
import scipy.io.wavfile as sciwav
from tqdm import tqdm
import json
from utils import get_recursive_files
import pathlib
import ctypes
from numpy.ctypeslib import ndpointer

In [56]:
lib_path = "./libfft.so"
lib = ctypes.cdll.LoadLibrary("./libfft.so")
fun = lib.get_footprint
fun.restype = None
fun.argtypes = [ndpointer(ctypes.c_int32, flags="C_CONTIGUOUS"),
                ctypes.c_size_t,
                ndpointer(ctypes.c_float, flags="C_CONTIGUOUS")]

In [5]:
def getFilesInFloder(folderPath):
    onlyfiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f))]
    return onlyfiles

def getDirsInFolder(baseDirPath):
    onlySubDirs = [d for d in listdir(baseDirPath) if isdir(join(baseDirPath, d))]
    return onlySubDirs

In [126]:
def read_and_convert_wav_to_16K_SR(wav_file):
    try:
        std_sr = 16000
        sr, sig = sciwav.read(wav_file)
        if sig.shape[0] < sig.size:
            sig = sig[0]
            print("\n{} is channel 2".format(wav_file))
        return sig.astype(np.int32)
    except:
        print("Error occured in read and convert wav to ndarray in file {}".format(wav_file))

In [7]:
# def read_and_convert_wav_to_16K_SR(wav_file):
#     sig, sr = librosa.load(wav_file, mono=True, sr=16000)
#     return sig

In [8]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [9]:
middleFreq = [16, 20, 26, 36, 48, 60, 80, 101, 256, 353]
inband = 4
bandnum_ = 10
y_length_=513
def genICFilterMatrix(bandnum, y_length, mid_freq_matrix):
    ret_icfilter = np.zeros((bandnum, y_length),dtype=float)
    for i in range(bandnum):
        if i == 8:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        elif i == 9:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        else:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-20*12*abs(np.log10(15.625*(j)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
    
    return ret_icfilter

In [7]:
def gen_reduced_icfb_from_FB(ic_filter, cut_portion):
    save_path = "./icfb_output/reduced_icfb_20200806.mat"
    if not os.path.exists("./icfb_output"):
        os.mkdir("./icfb_output")
    new_array = np.zeros((10,513))
    len_row = ic_filter.shape[0]
    len_col = ic_filter.shape[1]
    for i in range(len_row):
        for j in range(len_col):
            if ic_filter[i][j] < cut_portion:
                new_array[i][j] = 0.0
            else:
                new_array[i][j] = ic_filter[i][j]
    new_dict = {"icfb":new_array}
    spio.savemat(save_path, new_dict, oned_as='col')
    return new_array

In [8]:
# _icfb = genICFilterMatrix(bandnum_, y_length_, middleFreq)
# icfb_dict = {"icfb":_icfb}
# spio.savemat("./MainSrc/Drawing/melfb_output/icfb2.mat",icfb_dict,oned_as='col')

In [12]:
# melfb = spio.loadmat("./MainSrc/Drawing/melfb_output/fb_a_prime.mat")["melfb"]
lbl_len=513
sample_num = 40
# def gen_1x40_dimension_mat_array(s1,s2,s3,s4):
#     s_fft_1 = np.fft.fft(s1,n=1024,axis=0) 
#     s_fft_2 = np.fft.fft(s2,n=1024,axis=0) 
#     s_fft_3 = np.fft.fft(s3,n=1024,axis=0) 
#     s_fft_4 = np.fft.fft(s4,n=1024,axis=0) 

#     s_fft_1_513 = np.split(s_fft_1,[0,513],axis=0)[1]
#     s_fft_2_513 = np.split(s_fft_2,[0,513],axis=0)[1]
#     s_fft_3_513 = np.split(s_fft_3,[0,513],axis=0)[1]
#     s_fft_4_513 = np.split(s_fft_4,[0,513],axis=0)[1]
    
#     # there is one error element in every array in position 9 of each array
#     x1 = np.log(np.matmul(melfb,(np.abs(s_fft_1_513)**2)) + 0.00000001)
#     x2 = np.log(np.matmul(melfb,(np.abs(s_fft_2_513)**2)) + 0.00000001)
#     x3 = np.log(np.matmul(melfb,(np.abs(s_fft_3_513)**2)) + 0.00000001)
#     x4 = np.log(np.matmul(melfb,(np.abs(s_fft_4_513)**2)) + 0.00000001)
    
#     x= np.array([x1,x2,x3,x4],dtype="double").reshape(1,sample_num)
#     x_40 = x[0:40]
    
#     max_ele = np.amax(x_40,axis=1)
#     min_ele = np.amin(x_40,axis=1)
#     sum_of_x = np.sum(x_40)
    
#     x_normalize = (x_40-min_ele)/(max_ele-min_ele+0.0001)
#     return x_normalize

In [44]:
tmp_len= 1024-400
padding_zeros_2 = np.array(np.zeros(tmp_len,dtype=np.int))
ori_ary = np.array([1,2,3,4,5,6])
print(ori_ary.shape)
# tmp_ary = np.array(10,dtype=np.int)
# tmp = np.random.rand(np.random.randint(1, 100))
ori_ary.resize(len(ori_ary) + tmp_len) # <- ret is not referred to by anything else,
                                    #    so this works
ori_ary[-tmp_len:] = padding_zeros_2
print(ori_ary)

(6,)
[1 2 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0

In [47]:
def padding_ndarray(ori_ary, padding_ary, size):
    ret_ary = np.resize(ori_ary,new_shape=len(ori_ary) + size)
    ret_ary[-size:] = padding_ary
    return ret_ary

In [68]:
def test_padding_ndarray():
    rand1 = np.random.randint(100,size=400)
    print(type(rand1))
    print(rand1.shape)
    zeros = np.array(np.zeros(1024-400,dtype=np.int32))
    ary3 = padding_ndarray(rand1, zeros, 1024-400)
    print(ary3)

In [69]:
test_padding_ndarray()

<class 'numpy.ndarray'>
(400,)
[76 95 70 ...  0  0  0]


In [125]:
pad_len = 1024-400
padding_zeros = np.array(np.zeros(pad_len,dtype=np.int))
bandnum=10
# print(padding_zeros)
def gen_1x40_dimension_mat_array_cfft(s1,s2,s3,s4):
    s1_prime = padding_ndarray(s1,padding_zeros,pad_len)
    s2_prime = padding_ndarray(s2,padding_zeros,pad_len)
    s3_prime = padding_ndarray(s3,padding_zeros,pad_len)
    s4_prime = padding_ndarray(s4,padding_zeros,pad_len)
    out1 = np.empty((10),dtype=np.float32)
    out2 = np.empty((10),dtype=np.float32)
    out3 = np.empty((10),dtype=np.float32)
    out4 = np.empty((10),dtype=np.float32)
    fun(s1_prime,bandnum,out1)
    fun(s2_prime,bandnum,out2)
    fun(s3_prime,bandnum,out3)
    fun(s4_prime,bandnum,out4)
    x= np.array([out1,out2,out3,out4],dtype="double").reshape(1,sample_num)
    x_40 = x[0:40]
    
    max_ele = np.amax(x_40,axis=1)
    min_ele = np.amin(x_40,axis=1)
    sum_of_x = np.sum(x_40)

    x_normalize = (x_40-min_ele)/(max_ele-min_ele+0.000001)
    return x_normalize

In [88]:
def test_gen_1x40_dimension_mat_array_cfft():
    rand1 = np.random.randint(100,size=400,dtype=np.int32)
    rand2 = np.random.randint(100,size=400,dtype=np.int32)
    rand3 = np.random.randint(100,size=400,dtype=np.int32)
    rand4 = np.random.randint(100,size=400,dtype=np.int32)
    ary4 = gen_1x40_dimension_mat_array_cfft(rand1,rand2,rand3,rand4)
    print(ary4)

In [90]:
# test_gen_1x40_dimension_mat_array_cfft()

In [119]:
def perform_speech_feature_extraction_2(files):
    #ret_array = np.empty([s_len,40],dtype=float)
    all_footprints = np.array([])
    all_labels = np.array([])
    startPoint = 1600
    stepQuantity = 400
    local_less_norm_count = 0
    local_greater_norm_count = 0
    for file in tqdm(files):
        test_wav_array = read_and_convert_wav_to_16K_SR(file[0])
        s_len = int(400*(np.floor(len(test_wav_array)/400)))+1
        lblAry = np.array([file[1]])
        lblValue = int(np.array([file[1]]))
        for i in range(startPoint,s_len,stepQuantity):
            idx1 = i-1600
            idx2 = i-1200
            idx3 = i-800
            idx4 = i-400
            if lblValue == 1:
                norm_value = np.log(np.linalg.norm(test_wav_array[idx1:i])/file_norm + 0.0001)
                if norm_value > -3:
                    local_greater_norm_count += 1
                    s1 = test_wav_array[idx1:idx2] #0-399
                    s2 = test_wav_array[idx2:idx3] #400-799run_test_main,number
                    s3 = test_wav_array[idx3:idx4] #800-1199
                    s4 = test_wav_array[idx4:i]    #1200-1599
                    tmp_x_40 = gen_1x40_dimension_mat_array(s1,s2,s3,s4)
                    all_footprints = np.vstack((all_footprints, tmp_x_40[0])) if all_footprints.size else tmp_x_40[0]
                    all_labels = np.vstack((all_labels, lblAry)) if all_labels.size else lblAry
                else:
                    local_less_norm_count += 1
            else:
                local_greater_norm_count += 1
                s1 = test_wav_array[idx1:idx2] #0-399
                s2 = test_wav_array[idx2:idx3] #400-799run_test_main,number
                s3 = test_wav_array[idx3:idx4] #800-1199
                s4 = test_wav_array[idx4:i]    #1200-1599
                tmp_x_40 = gen_1x40_dimension_mat_array(s1,s2,s3,s4)
                all_footprints = np.vstack((all_footprints, tmp_x_40[0])) if all_footprints.size else tmp_x_40[0]
                all_labels = np.vstack((all_labels, lblAry)) if all_labels.size else lblAry
    return all_footprints, all_labels, local_greater_norm_count, local_less_norm_count

In [118]:
def perform_speech_feature_extraction(files):
    #ret_array = np.empty([s_len,40],dtype=float)
    tmpdatalist = []
    tmplbllist = []
    startPoint = 1600
    stepQuantity = 400
    local_less_norm_count = 0
    local_greater_norm_count = 0
    for file in tqdm(files):
        print("current processing file {}".format(file))
        test_wav_array = read_and_convert_wav_to_16K_SR(file[0])
        file_norm = np.linalg.norm(test_wav_array)
        s_len = int(400*(np.floor(len(test_wav_array)/400))) + 1
        lblValue = int(np.array([file[1]]))
        for i in range(startPoint,s_len,stepQuantity):
            idx1 = i-1600
            idx2 = i-1200
            idx3 = i-800
            idx4 = i-400
            if lblValue == 1:
                norm_value = np.log(np.linalg.norm(test_wav_array[idx1:i])/file_norm + 0.0001)
                if norm_value > -3:
                    local_greater_norm_count += 1
                    s1 = test_wav_array[idx1:idx2] #0-399
                    s2 = test_wav_array[idx2:idx3] #400-799 run_test_main,number
                    s3 = test_wav_array[idx3:idx4] #800-1199
                    s4 = test_wav_array[idx4:i]    #1200-1599
                    tmp_x_40 = gen_1x40_dimension_mat_array_cfft(s1,s2,s3,s4)
                    tmpdatalist.append(tmp_x_40[0])
                    tmplbllist.append(lblValue * 1.0)
                else:
                    local_less_norm_count += 1
            else:
                local_greater_norm_count += 1
                s1 = test_wav_array[idx1:idx2] #0-399
                s2 = test_wav_array[idx2:idx3] #400-799 run_test_main,number
                s3 = test_wav_array[idx3:idx4] #800-1199
                s4 = test_wav_array[idx4:i]    #1200-1599
                tmp_x_40 = gen_1x40_dimension_mat_array_cfft(s1,s2,s3,s4)
                tmpdatalist.append(tmp_x_40[0])
                tmplbllist.append(lblValue * 1.0)
    return tmpdatalist, tmplbllist, local_greater_norm_count, local_less_norm_count

In [120]:
def getFileList_Test():
    tmpTestDir = "../../Speech_DataSets/tmp/"
    tmpfiles = get_recursive_files(tmpTestDir)
    file_lst = []
    for t_f in tmpfiles:
        if t_f.endswith(".wav"):
            file_lst.append([t_f,0])
    ext_data, ext_lbl = perform_speech_feature_extraction(tmpfiles,0)
    ext_data_2, ext_lbl_2 = perform_speech_feature_extraction_2(file_lst)
    print("extraction 1 data shape is {}".format(len(ext_data)))
    print("extraction 2 data shape is {}".format(ext_data_2.shape))

In [121]:
# getFileList_Test()

In [122]:
def getFileList(file_src_dir=None):
#     noiseSrcDir = "../../Speech_DataSets/wavDataset20200807/noise"
#     speechSrcDir = "../../Speech_DataSets/wavDataset20200807/speech/"
    noiseSrcDir = "../../../../../Speech_DataSets/wavDataset20200807/noise"
    speechSrcDir = "../../../../../Speech_DataSets/wavDataset20200807/speech/"
    noise_files = get_recursive_files(noiseSrcDir)
    speech_files = get_recursive_files(speechSrcDir)
    file_set = []
    for n_f in noise_files:
        if n_f.endswith(".wav"):
            file_set.append([n_f,0])
    for s_f in speech_files:
        if s_f.endswith(".wav"):
            file_set.append([s_f,1])
    file_ary = np.array(file_set)
    np.random.shuffle(file_ary)
    return file_ary

In [129]:
def run_main():
    file_lists = getFileList()
    datalist, lbllist, pass_count, loss_count = perform_speech_feature_extraction(file_lists)
#     print(len(datalist))
#     print(len(lbllist))
    print("datalist length is {}".format(len(datalist)))
    print("pass count is {}".format(pass_count))
    print("loss count is {}".format(loss_count))
    training_mat_dict = {"x_data":datalist}
    training_lbl_mat_dict = {"y_data":lbllist}
    
    spio.savemat("./trainData_cfft/trainData_fbe_wdataset20200807_20200915.mat",training_mat_dict,oned_as='column')
    spio.savemat("./trainData_cfft/trainLabel_fbe_wdataset20200807_20200915.mat",training_lbl_mat_dict,oned_as='row')
    print("NCTU training mat files saved. |^_^| ")

In [130]:
run_main()

  0%|          | 0/388 [00:00<?, ?it/s]

current processing file ['../../../../../Speech_DataSets/wavDataset20200807/noise/NNoise_mod0716/M1_⌐τ«α2.dat.wav'
 '0']


  0%|          | 1/388 [00:00<01:53,  3.41it/s]

current processing file ['../../../../../Speech_DataSets/wavDataset20200807/noise/NNoise_mod0716/M1_└úºJñOñj├÷¬∙.dat.wav'
 '0']


  1%|          | 2/388 [00:00<01:52,  3.43it/s]

...

current processing file ['../../../../../Speech_DataSets/wavDataset20200807/speech/MIR-1K/davidson_1_lyrics.wav'
 '1']


100%|█████████▉| 387/388 [02:16<00:00,  2.59it/s]

current processing file ['../../../../../Speech_DataSets/wavDataset20200807/speech/MIR-1K/yifen_3_lyrics.wav'
 '1']


100%|██████████| 388/388 [02:16<00:00,  2.84it/s]


datalist length is 188890
pass count is 188890
loss count is 208072
NCTU training mat files saved. |^_^| 


In [15]:
matfile = "./NCTU_TrainingData/nctuTrainData_Norm_red_icfb_009.mat"
mat_cnt = spio.loadmat(matfile)["x_data"]
print(mat_cnt[43403])

[0.80377606 0.6238854  0.7635766  0.80573604 0.8625157  0.81235928
 0.65905134 0.37587523 0.02184781 0.         0.82512304 0.62730566
 0.78644113 0.86210281 0.97739804 0.8882958  0.76357258 0.51179199
 0.19877121 0.18477867 0.79229399 0.60346704 0.7715273  0.8873823
 0.99998772 0.89292575 0.80773199 0.53946976 0.22180993 0.19420531
 0.80323529 0.56393243 0.761743   0.88027315 0.98323415 0.90284683
 0.82732483 0.49738049 0.06597012 0.04113396]


In [53]:
lblary = spio.loadmat("./NCTU_TrainingData/nctuTrainLabel_red_icfb_wdataset20200807_20200810.mat")["y_data"]
print(lblary.shape)
lbl_lst = []
for c in lblary:
    cc = int(c)*1.0
    lbl_lst.append(cc)
new_lbl_ary = np.array(lbl_lst)
new_lbl_ary = new_lbl_ary.reshape((1,len(new_lbl_ary)))
# print(new_lbl_ary.shape)
adict = {"y_data":new_lbl_ary}
spio.savemat("./NCTU_TrainingData/nctuTrainLabel_Norm_red_icfb_20200813.mat", adict, oned_as='col')
# save_dict = {"y_data":trans_lblary}
# spio.savemat("./NCTU_TrainingData/nctuTrainLabel_Norm_red_icfb_009.mat",save_dict, oned_as='col')

(1, 188623)


TypeError: only size-1 arrays can be converted to Python scalars

In [29]:
test_wav2 = "../../Speech_DataSets/wavDataset20200807/speech/MIR-1K/ABJones_1_lyrics.wav"
wav_sig, sr = librosa.load(test_wav2, mono=True, sr=16000)#sciwav.read(test_wav2)
sr2, wav_sig2 = sciwav.read(test_wav2)
print(len(wav_sig))
print(sr)
print(len(wav_sig2))
print(sr2)
# print(wav_sig)

981275
16000
981275
16000
